<a href="https://colab.research.google.com/github/Hana1234567890/exchange-rate/blob/main/%E6%9C%9F%E6%9C%ABfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##安裝需要的套件
!pip install line-bot-sdk
!pip install flask_ngrok
!pip install flask pyngrok
!pip install yfinance
!pip install imgurpython
!pip install matplotlib
!pip install mplfinance
!pip install gnews
#!pip install newsapi-python
#!pip install python-dotenv
!pip install requests
!pip install pandas
!pip install --upgrade line-bot-sdk


#讓ngrok套件連接雲端硬碟
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok
!/ngrok authtoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.0/768.0 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imgurpython: filename=imgurpython-1.1.7-py3-none-any.whl size=12555 sha256=7933bd7805e829ba650e89bc912dfef8a7c48c296e6f72149dcb7cac3377768c
  Stored in directory: /root/.cache/pip/wheels/3d/f0/cc/4bf6a0aff9619e3da19eeeaa369d98d4ee378a0bc217aa45bf
Successfully built imgurpython
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 26.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=a82909bf0839842d556ac528dbfd5a680f3c9fb0307c24ca9e65d4028b58c63a
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be

In [ ]:


##與Line主程式合併##
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    PushMessageRequest,
    TextMessage,
    ImageMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)


from pyngrok import ngrok,conf #用來創造虛擬網址 Import：載入
import yfinance as yf     #用來抓金融數據
import datetime         #用來擷取日期跟時間
import matplotlib #用來畫圖的套件
import mplfinance as mpf #用來畫財金圖的套件
import pandas as pd

from gnews import GNews
from linebot import LineBotApi
from linebot.models import FlexSendMessage





#########################################################################################
app = Flask(__name__)
conf.get_default().auth_token = "23vO2SNPPGqDHyNuDXo8zYHK52C_LwsqCGTL5RJY4LtR6q7V"
public_url = ngrok.connect(5000)
print('##########################################')
print("你的虛擬主機網址：",public_url)
print('##########################################')
######################LINE的群組帳號######################################################
configuration = Configuration(access_token='3YMNpAHGuBpRruzfSFjpf18n67Zl77O7LP5c59JPlqTeCf6VU1rTo8+Zl1RQKBUIDUTe+spsT+U1iLtn6Tlr47BctYXkSkD2Kc+5hHzxfmogjbDZ067LZl/dlMXDJ2MXVxBWXMJZug5p6Ws8m0PpEQdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('6f5a51a705ee8103ed9103791eb2e270')
######################上傳圖片的群組帳號##################################################
import matplotlib
matplotlib.use('Agg')
import datetime
from imgurpython import ImgurClient
####Imgur網站的群組帳密
client_id = '5cdd69f729edae5'
client_secret = '809fb7b8db31a466f78f11bcaae57fe83e3fdec8'
album_id = '股票趨勢'
access_token = 'b6c603d9b0915739ad42e5345088d9ca555cf389'
refresh_token = '30c31a1ca1020e765fdd7549aea2059d018dab84'
##########################################################################################
對照表={'元大台灣50':'0050.TW',
    '富邦台50':'006208.TW',
    '台積電':'2330.TW',
    '聯發科':'2454.TW',
    '鴻海':'2317.TW',
    '中鋼':'2002.TW',

    }
###################自己創造的新功能(函數)#################################################
def 畫日交易圖(股票代碼):
  df = yf.download(股票代碼,period='1d',interval='1m') #擷取1天股價資料，頻率為1分鐘
  df.columns = df.columns.droplevel(1)   #刪除第一行的資料
  df = df.dropna()             #把沒有資料的數據刪除
  mc = mpf.make_marketcolors(up='r',down='g',inherit=True) #設定漲跌的顏色
  s = mpf.make_mpf_style(base_mpf_style='yahoo',marketcolors=mc) #設置圖片的版型為yahoo
  save_path = '/content/drive/MyDrive/ngrok-ssh/' + 股票代碼 + '.png' #設定存檔位置
  kwargs = dict(type='candle',mav=(5,20,60),volume=True,figscale=1,title=str(股票代碼),savefig=save_path) #candle:蠟燭圖、5筆20筆60筆資料的移動平均
  mpf.plot(df,**kwargs,style=s)
  return showImgur(股票代碼)
def showImgur(fileName): #定義(創立)一個叫showImgur的函數
        client= ImgurClient(client_id, client_secret, access_token, refresh_token)  #連接imgur
        config = {
            # 'album': album_id, # 相簿名稱
            'name': fileName, # 圖片名稱
            'title': fileName, # 圖片標題
            'description': str(datetime.date.today())} # 備註:輸入日期時間
        try: # 開始上傳檔案
            print("上傳圖片中 ")
            imgurl = client.upload_from_path(fileName+'.png', config=config, anon=False)['link']
            print("圖片上傳完成 ")
        except :
            imgurl = 'https://i.imgur.com/RFmkvQX.jpg'   # 如果失敗回傳"失敗"這張圖
            print("無法上傳圖片 ")
        return imgurl


def 抓新聞(關鍵字):
  google_新聞=GNews(language='zh',country='TW')
  新聞列表=google_新聞.get_news(關鍵字)
  新聞彙總=[]
  for 新聞 in 新聞列表:
    新聞彙總.append({
        '標題':新聞['title'],
        '連結':新聞['url'],
        '時間':新聞['published date'],
        '摘要':['description']
    })
  df=pd.DataFrame(新聞彙總)
  if '時間' in df.columns:
    df['時間']=pd.to_datetime(df['時間'],errors='coerce')
    df=df.sort_values(by='時間',ascending=False)
  return df.head(3)

def 傳送關鍵字新聞(關鍵字,user_id):
  df = 抓新聞(關鍵字)
  for i in range(len(df)):
    標題=df.iloc[i]['標題'][:20]
    連結=df.iloc[i]['連結']
    flex_message = FlexSendMessage(alt_text="最新關鍵字新聞",
              contents={"type":"bubble",
                    "body":{"type": "box","layout": "vertical","contents": [{"type": "text","text": 標題},
                    {"type": "button","action":{"type": "uri","label": "查看新聞","uri": 連結},
                              "style": "primary",
                              "color": "#1DB446"}]}})
    line_bot_api=LineBotApi(configuration.access_token) #初始化lineBotApi
    line_bot_api.push_message(user_id, flex_message)

import requests
import pandas as pd
import matplotlib.pyplot as plt
def 查看黃金價格():

  try:
    response=requests.get('https://rate.bot.com.tw/gold/chart/year/TWD',timeout=10)
    response.raise_for_status()
    with open('gold_chart.html', 'w' ,encoding='utf-8') as f:
        f.write(response.text)
    print("成功抓取最新資料")
  except requests.exceptions.Timeout:
    print("請求超時，請使用本地檔案")
  except requests.exceptions.RequestException as e:
    print(f"無法抓取資料，錯誤: {e}")
    print("使用本地檔案")
  #讀取本地檔案
  try:
    df=pd.read_html('gold_chart.html')[0]
    df=df.rename(columns={'日期' : 'Date','本行賣出價格':'Price'})
    df['Date']=pd.to_datetime(df['Date'], format="%Y/%m/%d")
    df.set_index('Date',inplace=True)
    df.sort_index(inplace=True)
    df['Price'].plot(title="Gold_Price")
    plt.savefig('黃金價格.png')
    print("圖表保存完成")
    return showImgur('黃金價格'),df

  except Exception as e:
    print(f"無法抓取資料，錯誤:{e}")
    return None

#即時外匯
def 美金即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入美金=df.loc['USD','現金.1']
  return 現金買入美金

def 美金匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/USD')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='USD Exchange Rate')
  plt.savefig('美金歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('美金歷史外匯'),df


def 日圓即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入日圓=df.loc['JPY','現金.1']
  return 現金買入日圓

def 日圓匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/JPY')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='JPY Exchange Rate')
  plt.savefig('日圓歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('日圓歷史外匯'),df


def 港幣即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入港幣=df.loc['HKD','現金.1']
  return 現金買入港幣

def 港幣匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/HKD')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='HKD Exchange Rate')
  plt.savefig('港幣歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('港幣歷史外匯'),df


def 英鎊即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入英鎊=df.loc['GBP','現金.1']
  return 現金買入英鎊

def 英鎊匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/GBP')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='GBP Exchange Rate')
  plt.savefig('英鎊歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('英鎊歷史外匯'),df


#澳幣
def 澳幣即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入澳幣=df.loc['AUD','現金.1']
  return 現金買入澳幣

def 澳幣匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/AUD')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='AUD Exchange Rate')
  plt.savefig('澳幣歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('澳幣歷史外匯'),df


#加拿大幣
def 加拿大幣即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入加拿大幣=df.loc['CAD','現金.1']
  return 現金買入加拿大幣

def 加拿大幣匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/CAD')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='CAD Exchange Rate')
  plt.savefig('加拿大幣歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('加拿大幣歷史外匯'),df

# 新加坡幣(SGD)
def 新加坡幣即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入新加坡幣=df.loc['SGD','現金.1']
  return 現金買入新加坡幣

def 新加坡幣匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/SGD')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='SGD Exchange Rate')
  plt.savefig('新加坡幣歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('新加坡幣歷史外匯'),df


#瑞士法郎 (CHF)
def 瑞士法郎即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入瑞士法郎=df.loc['CHF','現金.1']
  return 現金買入瑞士法郎

def 瑞士法郎匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/CHF')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='CHF Exchange Rate')
  plt.savefig('瑞士法郎歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('瑞士法郎歷史外匯'),df


#南非幣 (ZAR)
def 南非幣即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入南非幣=df.loc['ZAR','現金.1']
  return 現金買入南非幣

def 南非幣匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/ZAR')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='ZAR Exchange Rate')
  plt.savefig('南非幣歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('南非幣歷史外匯'),df


#瑞典幣 (SEK)
def 瑞典幣即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入瑞典幣=df.loc['SEK','現金.1']
  return 現金買入瑞典幣

def 瑞典幣匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/SEK')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='SEK Exchange Rate')
  plt.savefig('瑞典幣歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('瑞典幣歷史外匯'),df


#紐元 (NZD)
def 紐元即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入紐元=df.loc['NZD','現金.1']
  return 現金買入紐元

def 紐元匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/NZD')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='NZD Exchange Rate')
  plt.savefig('紐元歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('紐元歷史外匯'),df


#泰幣 (THB)
def 泰幣即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入泰幣=df.loc['THB','現金.1']
  return 現金買入泰幣

def 泰幣匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/THB')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='THB Exchange Rate')
  plt.savefig('泰幣歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('泰幣歷史外匯'),df


#菲國比索 (PHP)
def 菲國比索即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入菲國比索=df.loc['PHP','現金.1']
  return 現金買入菲國比索

def 菲國比索匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/PHP')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='PHP Exchange Rate')
  plt.savefig('菲國比索歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('菲國比索歷史外匯'),df

#印尼幣 (IDR)
def 印尼幣即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入印尼幣=df.loc['IDR','現金.1']
  return 現金買入印尼幣

def 印尼幣匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/IDR')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='IDR Exchange Rate')
  plt.savefig('印尼幣歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('印尼幣歷史外匯'),df

#歐元 (EUR)
def 歐元即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入歐元=df.loc['EUR','現金.1']
  return 現金買入歐元

def 歐元匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/EUR')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='EUR Exchange Rate')
  plt.savefig('歐元歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('歐元歷史外匯'),df

#韓元 (KRW)
def 韓元即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入韓元=df.loc['KRW','現金.1']
  return 現金買入韓元

def 韓元匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/KRW')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='KRW Exchange Rate')
  plt.savefig('韓元歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('韓元歷史外匯'),df

#越南盾 (VND)
def 越南盾即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入越南盾=df.loc['VND','現金.1']
  return 現金買入越南盾

def 越南盾匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/VND')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='VND Exchange Rate')
  plt.savefig('越南盾歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('越南盾歷史外匯'),df

#馬來幣 (MYR)
def 馬來幣即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入馬來幣=df.loc['MYR','現金.1']
  return 現金買入馬來幣

def 馬來幣匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/MYR')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='MYR Exchange Rate')
  plt.savefig('馬來幣歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('馬來幣歷史外匯'),df

#人民幣 (CNY)
def 人民幣即時匯率():
  df=pd.read_csv('https://rate.bot.com.tw/xrt/flcsv/0/day')
  現金買入人民幣=df.loc['CNY','現金.1']
  return 現金買入人民幣

def 人民幣匯率變化圖():
  df=pd.read_html('https://rate.bot.com.tw/xrt/quote/l6m/CNY')[0]
  df.columns=['掛牌日期','幣別','現金買入','現金賣出','即期買入','即期賣出','0','1','2']
  df=df[['掛牌日期','現金賣出']]
  df['掛牌日期']=pd.to_datetime(df['掛牌日期'],format='%Y/%m/%d')
  df.set_index('掛牌日期',inplace=True)
  df.sort_index(inplace=True)
  df['現金賣出'].plot(title='CNY Exchange Rate')
  plt.savefig('人民幣歷史外匯.png')
                    #避免圖片重複繪製
  print("圖表保存完成")
  return showImgur('人民幣歷史外匯'),df

#換算
def 即時匯率(幣別):
    """
    查詢即時匯率。
    :param 幣別: 幣別代碼，例如 'KRW'。
    :return: 匯率數值，若無法查詢則返回 None。
    """
    try:
        url = 'https://rate.bot.com.tw/xrt/flcsv/0/day'
        df = pd.read_csv(url)

        # 檢查有無該幣別
        if 幣別 not in df.index:
            return None

        # 獲取現金買入匯率
        匯率值 = df.loc[幣別, '現金.1']
        if pd.isna(匯率值) or 匯率值 == 0:
            return None

        return float(匯率值)
    except Exception as e:
        print(f"查詢即時匯率錯誤: {e}")
        return None


def 匯率換算(來源幣別, 目標幣別, 金額):
    """
    匯率換算功能。
    :param 來源幣別: 來源幣別代碼，例如 "USD"。
    :param 目標幣別: 目標幣別代碼，例如 "TWD"。
    :param 金額: 需要換算的金額。
    :return: 換算後的金額或錯誤訊息。
    """
    if 來源幣別 == "TWD":
        來源匯率 = 1
    else:
        來源匯率 = 即時匯率(來源幣別)
        if 來源匯率 is None:
            return f"無法查詢來源幣別 {來源幣別} 的即時匯率。"

    if 目標幣別 == "TWD":
        目標匯率 = 1
    else:
        目標匯率 = 即時匯率(目標幣別)
        if 目標匯率 is None:
            return f"無法查詢目標幣別 {目標幣別} 的即時匯率。"

    # 計算匯率換算結果
    換算結果 = 金額 * (來源匯率 / 目標匯率)
    return round(換算結果, 2)


幣別對應 = {
    '台幣': 'TWD',
    '美金': 'USD',
    '日幣': 'JPY',
    '港幣': 'HKD',
    '歐元': 'EUR',
    '英鎊': 'GBP',
    '澳幣': 'AUD',
    '加拿大幣': 'CAD',
    '新加坡幣': 'SGD',
    '瑞士法郎': 'CHF',
    '南非幣': 'ZAR',
    '瑞典幣': 'SEK',
    '紐元': 'NZD',
    '泰幣': 'THB',
    '菲國比索': 'PHP',
    '印尼幣': 'IDR',
    '韓元': 'KRW',
    '越南盾': 'VND',
    '馬來幣': 'MYR',
    '人民幣': 'CNY',
}


##########################################################################################
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

##########主程式########
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        輸入的文字=event.message.text
        user_id=event.source.user_id
        if '查詢' in 輸入的文字:
          if '股價' in 輸入的文字:
            公司簡稱 = 輸入的文字[2:-2]
            股票代碼 = 對照表.get(公司簡稱,None)
            if 股票代碼 is not None:
              line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在處理您的請求，請稍後...")]))
              df = yf.download(股票代碼,start='2021-01-01',end=datetime.date.today())
              df.columns = df.columns.droplevel(1)
              df=df.dropna()
              最新開盤價 = str(round(df.iloc[-1]['Open'],2))
              最新收盤價 = str(round(df.iloc[-1]['Close'],2))
              漲跌幅 = str(round((df.iloc[-1]['Close']-df.iloc[-1]['Open'])/df.iloc[-1]['Open']*100,2))
              五天平均收盤價 = str(round(df.iloc[-5:]['Close'].mean(),2))
              五天平均交易量 = str(int(df.iloc[-5:]['Volume'].mean()))
              今天的日期 = str(df.index[-1])[:4]+'年'+str(df.index[-1])[5:7]+'月'+str(df.index[-1])[8:10]+'日'
              推播的文字 = 今天的日期+公司簡稱+'開盤價為'+最新開盤價+'元，收盤價為'+最新收盤價+'元，漲跌幅為'+漲跌幅+'%。'+'5日平均收盤價為'+五天平均收盤價+'元，交易量為'+五天平均交易量+'股'
              圖片的網址 = 畫日交易圖(股票代碼)
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            else:
              line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="股票代碼並未在資料庫，以使用公司代碼為您處理")]))
              股票代碼 = 輸入的文字[2:-2]
              df = yf.download(股票代碼,start='2021-01-01',end=datetime.date.today())
              df.columns = df.columns.droplevel(1)
              df=df.dropna()
              最新開盤價 = str(round(df.iloc[-1]['Open'],2))
              最新收盤價 = str(round(df.iloc[-1]['Close'],2))
              漲跌幅 = str(round((df.iloc[-1]['Close']-df.iloc[-1]['Open'])/df.iloc[-1]['Open']*100,2))
              五天平均收盤價 = str(round(df.iloc[-5:]['Close'].mean()))
              五天平均交易量 = str(int(df.iloc[-5:]['Volume'].mean()))
              今天的日期 = str(df.index[-1])[:4]+'年'+str(df.index[-1])[5:7]+'月'+str(df.index[-1])[8:10]+'日'
              推播的文字 = 今天的日期+公司簡稱+'開盤價為'+最新開盤價+'元，收盤價為'+最新收盤價+'元，漲跌幅為'+漲跌幅+'%。'+'5日平均收盤價為'+五天平均收盤價+'元，交易量為'+五天平均交易量+'股'
              圖片的網址 = 畫日交易圖(股票代碼)
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))

          elif '最新新聞' in 輸入的文字:
            關鍵字=輸入的文字[2:-4]
            傳送關鍵字新聞(關鍵字,user_id)

          elif '黃金價格' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新黃金標價，請稍後...")]))
            try:
              圖片的網址, df=查看黃金價格()
              今天的日期 = str(df.index[-1])[:4]+'年'+str(df.index[-1])[5:7]+'月'+str(df.index[-1])[8:10]+'日'
              最新價格 = str(df.iloc[-1]['Price'])
              推播的文字 = f'老闆,您好。{今天的日期}最新黃金價格為每公克{最新價格}元'
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message =(f"抱歉，無法查詢黃金價格:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))


          elif '美金' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新美金匯率，請稍後...")]))
            try:
              美金的即時價格=美金即時匯率()
              最新價格 = str(美金的即時價格)
              推播的文字 = f'老闆,您好。最新美金匯率為{最新價格}元'
              圖片的網址, df=美金匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢美金匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '日圓' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新日圓匯率，請稍後...")]))
            try:
              日圓的即時價格=日圓即時匯率()
              最新價格 = str(日圓的即時價格)
              推播的文字 = f'老闆,您好。最新日圓匯率為{最新價格}元'
              圖片的網址, df=日圓匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢日圓匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '港幣' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新港幣匯率，請稍後...")]))
            try:
              港幣的即時價格=港幣即時匯率()
              最新價格 = str(港幣的即時價格)
              推播的文字 = f'老闆,您好。最新港幣匯率為{最新價格}元'
              圖片的網址, df=港幣匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢港幣匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '英鎊' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新英鎊匯率，請稍後...")]))
            try:
              英鎊的即時價格=英鎊即時匯率()
              最新價格 = str(英鎊的即時價格)
              推播的文字 = f'老闆,您好。最新英鎊匯率為{最新價格}元'
              圖片的網址, df=英鎊匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢英鎊匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '澳幣' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新澳幣匯率，請稍後...")]))
            try:
              澳幣的即時價格=澳幣即時匯率()
              最新價格 = str(澳幣的即時價格)
              推播的文字 = f'老闆,您好。最新澳幣匯率為{最新價格}元'
              圖片的網址, df=澳幣匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢澳幣匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '加拿大幣' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新加拿大幣匯率，請稍後...")]))
            try:
              加拿大幣的即時價格=加拿大幣即時匯率()
              最新價格 = str(加拿大幣的即時價格)
              推播的文字 = f'老闆,您好。最新加拿大幣匯率為{最新價格}元'
              圖片的網址, df=加拿大幣匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢加拿大幣匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '新加坡幣' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新新加坡幣匯率，請稍後...")]))
            try:
              新加坡幣的即時價格=新加坡幣即時匯率()
              最新價格 = str(新加坡幣的即時價格)
              推播的文字 = f'老闆,您好。最新新加坡幣匯率為{最新價格}元'
              圖片的網址, df=新加坡幣匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢新加坡幣匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '瑞士法郎' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新瑞士法郎匯率，請稍後...")]))
            try:
              瑞士法郎的即時價格=瑞士法郎即時匯率()
              最新價格 = str(瑞士法郎的即時價格)
              推播的文字 = f'老闆,您好。最新瑞士法郎匯率為{最新價格}元'
              圖片的網址, df=瑞士法郎匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢瑞士法郎匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '南非幣' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新南非幣匯率，請稍後...")]))
            try:
              南非幣的即時價格=南非幣即時匯率()
              最新價格 = str(南非幣的即時價格)
              推播的文字 = f'老闆,您好。最新南非幣匯率為{最新價格}元'
              圖片的網址, df=南非幣匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢南非幣匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '瑞典幣' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新瑞典幣匯率，請稍後...")]))
            try:
              瑞典幣的即時價格=瑞典幣即時匯率()
              最新價格 = str(瑞典幣的即時價格)
              推播的文字 = f'老闆,您好。最新瑞典幣匯率為{最新價格}元'
              圖片的網址, df=瑞典幣匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢瑞典幣匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '紐元' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新紐元匯率，請稍後...")]))
            try:
              紐元的即時價格=紐元即時匯率()
              最新價格 = str(紐元的即時價格)
              推播的文字 = f'老闆,您好。最新紐元匯率為{最新價格}元'
              圖片的網址, df=紐元匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢紐元匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '泰幣' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新泰幣匯率，請稍後...")]))
            try:
              泰幣的即時價格=泰幣即時匯率()
              最新價格 = str(泰幣的即時價格)
              推播的文字 = f'老闆,您好。最新泰幣匯率為{最新價格}元'
              圖片的網址, df=泰幣匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢泰幣匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '菲國比索' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新菲國比索匯率，請稍後...")]))
            try:
              菲國比索的即時價格=菲國比索即時匯率()
              最新價格 = str(菲國比索的即時價格)
              推播的文字 = f'老闆,您好。最新菲國比索匯率為{最新價格}元'
              圖片的網址, df=菲國比索匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢菲國比索匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '印尼幣' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新印尼幣匯率，請稍後...")]))
            try:
              印尼幣的即時價格=印尼幣即時匯率()
              最新價格 = str(印尼幣的即時價格)
              推播的文字 = f'老闆,您好。最新印尼幣匯率為{最新價格}元'
              圖片的網址, df=印尼幣匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢印尼幣元匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '歐元' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新歐元匯率，請稍後...")]))
            try:
              歐元的即時價格=歐元即時匯率()
              最新價格 = str(歐元的即時價格)
              推播的文字 = f'老闆,您好。最新歐元匯率為{最新價格}元'
              圖片的網址, df=歐元匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢歐元匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '韓元' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新韓元匯率，請稍後...")]))
            try:
              韓元的即時價格=韓元即時匯率()
              最新價格 = str(韓元的即時價格)
              推播的文字 = f'老闆,您好。最新韓元匯率為{最新價格}元'
              圖片的網址, df=韓元匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢韓元匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '越南盾' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新越南盾匯率，請稍後...")]))
            try:
              越南盾的即時價格=越南盾即時匯率()
              最新價格 = str(越南盾的即時價格)
              推播的文字 = f'老闆,您好。最新越南盾匯率為{最新價格}元'
              圖片的網址, df=越南盾匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢越南盾匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '歐元' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新歐元匯率，請稍後...")]))
            try:
              歐元的即時價格=歐元即時匯率()
              最新價格 = str(歐元的即時價格)
              推播的文字 = f'老闆,您好。最新歐元匯率為{最新價格}元'
              圖片的網址, df=歐元匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢歐元匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))


          elif '馬來幣' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新馬來幣匯率，請稍後...")]))
            try:
              馬來幣的即時價格=馬來幣即時匯率()
              最新價格 = str(馬來幣的即時價格)
              推播的文字 = f'老闆,您好。最新馬來幣匯率為{最新價格}元'
              圖片的網址, df=馬來幣匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢馬來幣匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))

          elif '人民幣' in 輸入的文字:

            line_bot_api.reply_message(ReplyMessageRequest(reply_token=event.reply_token,messages=[TextMessage(text="正在為您搜尋台灣銀行最新人民幣匯率，請稍後...")]))
            try:
              人民幣的即時價格=人民幣即時匯率()
              最新價格 = str(人民幣的即時價格)
              推播的文字 = f'老闆,您好。最新人民幣匯率為{最新價格}元'
              圖片的網址, df=人民幣匯率變化圖()
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=推播的文字),ImageMessage(originalContentUrl=圖片的網址,previewImageUrl=圖片的網址)]))
            except Exception as e:
              error_message = (f"抱歉，無法查詢人民幣匯率:{e}")
              line_bot_api.push_message(PushMessageRequest(to=user_id,messages=[TextMessage(text=error_message)]))



        if '換算' in 輸入的文字:
          try:

              if '到' in 輸入的文字:
                  # 拆解使用者輸入，提取來源幣別、目標幣別和金額
                  parts = 輸入的文字.replace("換算", "").split("到")
                  來源幣別中文 = parts[0].strip()
                  其他部分 = parts[1].strip().split()

                  # 確保分割後格式正確
                  if len(其他部分) < 2:
                      raise ValueError("輸入格式有誤，請確保包含目標幣別與金額。例如：換算台幣到美金 100")

                  目標幣別中文 = 其他部分[0].strip()
                  try:
                      金額 = float(其他部分[1].strip())
                      if 金額 <= 0:
                          raise ValueError("金額必須是正數，請重新輸入。")
                  except ValueError:
                      raise ValueError("金額必須是數字，請確認後重新輸入。")

                  # 將中文幣別名稱映射為幣別代碼
                  來源幣別 = 幣別對應.get(來源幣別中文)
                  目標幣別 = 幣別對應.get(目標幣別中文)

                  if not 來源幣別 or not 目標幣別:
                      未識別幣別 = []
                      if not 來源幣別:
                          未識別幣別.append(來源幣別中文)
                      if not 目標幣別:
                          未識別幣別.append(目標幣別中文)
                      raise ValueError(f"無法識別以下幣別：{', '.join(未識別幣別)}，請確認輸入是否正確。")

                  # 執行匯率換算
                  換算結果 = 匯率換算(來源幣別, 目標幣別, 金額)

                  # 判斷換算結果並回覆
                  if isinstance(換算結果, str):  # 換算結果是錯誤訊息
                      line_bot_api.reply_message(
                          ReplyMessageRequest(
                              reply_token=event.reply_token,
                              messages=[TextMessage(text=換算結果)]
                          )
                      )
                  else:  # 換算成功
                      推播的文字 = f"{金額} {來源幣別中文} 可換算為 {換算結果} {目標幣別中文}"
                      line_bot_api.reply_message(
                          ReplyMessageRequest(
                              reply_token=event.reply_token,
                              messages=[TextMessage(text=推播的文字)]
                          )
                      )
              else:
                  raise ValueError("換算格式錯誤，請輸入：換算[來源幣別]到[目標幣別] [金額]")

          except Exception as e:
              # 回覆錯誤訊息給使用者
              line_bot_api.reply_message(
                  ReplyMessageRequest(
                      reply_token=event.reply_token,
                      messages=[TextMessage(text=f"換算失敗：{e}")]
                  )
              )



if __name__ == "__main__":
    app.run()

##輸入關鍵字:查詢台積電股價、查詢1101.TW股價、查詢矽品最新新聞、查詢黃金價格、查詢美金匯率



##########################################
你的虛擬主機網址： NgrokTunnel: "https://7bc6-34-150-205-23.ngrok-free.app" -> "http://localhost:5000"
##########################################
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Jan/2025 15:38:58] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Jan/2025 15:39:15] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Jan/2025 15:39:27] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Jan/2025 15:39:43] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Jan/2025 15:39:53] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Jan/2025 15:40:03] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Jan/2025 15:40:13] "POST /callback HTTP/1.1" 200 -


In [ ]:
print(即時匯率('韓圜'))

None
